In [1]:
%%capture
!pip install yfinance

In [2]:
import numpy as np
import pandas as pd
import datetime

import yfinance as yf

import cvxopt as opt
from cvxopt import solvers, matrix

pd.options.display.float_format = '{:.6f}'.format

In [3]:
class Crypto:
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.close_prices = self.initialize_close_prices()

    def initialize_close_prices(self):
        close_prices = pd.DataFrame()

        # Get close prices for each ticker
        for ticker in self.tickers:
            crypto = yf.Ticker(ticker)
            close_price = crypto.history(start=self.start_date, end=self.end_date)['Close']
            close_prices[ticker] = close_price

        # Drop rows that contain missing values
        close_prices.dropna(inplace=True)

        # Convert the index to datetime objects with the desired format
        close_prices.index = pd.to_datetime(close_prices.index, format='%Y-%m-%d')

        # Extract the year-month-day from the datetime objects and use it as the new index
        close_prices.index = close_prices.index.strftime('%Y-%m-%d')

        return close_prices

    def get_close_prices(self):
        return self.close_prices

    def minimum_variance_portfolio_with_positive_weights(self, target_return=None):
        # Calculate daily returns
        returns = self.close_prices.pct_change().dropna()

        # Check if we can achieve desiable target_return
        mean_ret = returns.mean()
        min_ret, max_ret = mean_ret.min(), mean_ret.max()
        if target_return is not None and (target_return < min_ret or target_return > max_ret):
            raise ValueError(f'Inappropriate value of target return. It should be between {min_ret} and {max_ret}')

        # Calculate covariance matrix
        cov_matrix = returns.cov()
        
        # Convert covariance matrix to cvxopt matrix
        Sigma = opt.matrix(cov_matrix.values)

        # Create constraint matrices
        n = len(returns.columns)
        pbar = opt.matrix(np.ones(n))
        G = opt.matrix(np.vstack((-np.eye(n), np.eye(n))))  # negative and positive n x n identity matrices; Gx <= h
        h = opt.matrix(np.vstack((np.zeros((n, 1)), np.ones((n, 1)))))  # n x 1 matrices of zeros and ones
        A = opt.matrix(1.0, (1, n))  # Ax = b
        b = opt.matrix(1.0)

        if target_return is not None:
            # Add target return constraint: mu'x >= target_return
            mu = opt.matrix(mean_ret)
            G = opt.matrix(np.vstack((G, -mu.T)))
            h = opt.matrix(np.vstack((h, -target_return)))
    
        # Calculate efficient frontier weights using quadratic programming
        solvers.options['show_progress'] = False
        portfolio_weights = solvers.qp(Sigma, pbar, G, h, A, b)  #['x']

        # Check feasibility
        if portfolio_weights['status'] == 'optimal':
            portfolio_weights = np.array(portfolio_weights['x']).flatten().tolist()
            weight_map = pd.DataFrame({'Crypto': self.close_prices.columns, 
                                       'Daily_Return': mean_ret.values,
                                       'Std': np.sqrt(np.diag(cov_matrix)),
                                       'Weight': [float(i) for i in portfolio_weights]})
            return weight_map
        else:
            raise ValueError(f'Feasible solution not found for target return of {target_return}')

In [4]:
# Cryptocurrencies you want to include in the portfolio
tickers = ['BTC-USD', 'ETH-USD', 'LTC-USD', 'ADA-USD', 'DASH-USD', 'BNB-USD', 'BCH-USD', 'XLM-USD', 'XRP-USD', 'TRX-USD']

# Get the current date
now = datetime.datetime.now()

# Format the date in the required format
start_date = '2017-01-01'
end_date = now.strftime("%Y-%m-%d")

# Minimum variance portfolio with positive weights
Cryptos = Crypto(tickers, start_date, end_date)

In [5]:
Cryptos.minimum_variance_portfolio_with_positive_weights()

,Crypto,Daily_Return,Std,Weight
0,BTC-USD,0.001413,0.039764,0.909027
1,ETH-USD,0.002146,0.050410,0.000289
2,LTC-USD,0.001696,0.055446,0.000011
3,ADA-USD,0.003665,0.074276,0.000010
4,DASH-USD,0.000815,0.060553,0.014162
5,BNB-USD,0.004383,0.060262,0.014981
6,BCH-USD,0.001215,0.064957,0.000005
7,XLM-USD,0.002349,0.063424,0.000155
8,XRP-USD,0.002405,0.066465,0.061349
9,TRX-USD,0.004288,0.077559,0.000012


In [6]:
Cryptos.minimum_variance_portfolio_with_positive_weights(target_return=0.002)

,Crypto,Daily_Return,Std,Weight
0,BTC-USD,0.001413,0.039764,0.750800
1,ETH-USD,0.002146,0.050410,0.008021
2,LTC-USD,0.001696,0.055446,0.000264
3,ADA-USD,0.003665,0.074276,0.005049
4,DASH-USD,0.000815,0.060553,0.000409
5,BNB-USD,0.004383,0.060262,0.144681
6,BCH-USD,0.001215,0.064957,0.000107
7,XLM-USD,0.002349,0.063424,0.004775
8,XRP-USD,0.002405,0.066465,0.058931
9,TRX-USD,0.004288,0.077559,0.026962
